# Démonstration : Diffie-Hellman Key Exchange

**Objectifs** :
- Implémenter l'échange de clés Diffie-Hellman
- Comprendre le problème du logarithme discret
- Démontrer l'attaque Man-in-the-Middle
- Utiliser des courbes elliptiques (ECDH)

In [ ]:
from cryptography.hazmat.primitives.asymmetric import dh, ec
from cryptography.hazmat.primitives import hashes, serialization
from cryptography.hazmat.primitives.kdf.hkdf import HKDF
from cryptography.hazmat.backends import default_backend
import secrets

## 1. Diffie-Hellman classique (sur $\mathbb{Z}_p^*$)

**Protocole** :
1. Paramètres publics : groupe $G$ d'ordre $q$, générateur $g$
2. Alice choisit $a \xleftarrow{\$} \mathbb{Z}_q$, envoie $A = g^a$
3. Bob choisit $b \xleftarrow{\$} \mathbb{Z}_q$, envoie $B = g^b$
4. Alice calcule $K = B^a = g^{ab}$
5. Bob calcule $K = A^b = g^{ab}$
6. Clé partagée : $K = g^{ab}$

In [ ]:
# Génération des paramètres DH (groupe)
# En pratique, utiliser des paramètres standard (RFC 3526, RFC 7919)
print("Génération des paramètres DH (peut prendre quelques secondes)...")
parameters = dh.generate_parameters(generator=2, key_size=2048, backend=default_backend())

# Paramètres publics (p, g)
p = parameters.parameter_numbers().p
g = parameters.parameter_numbers().g

print(f"\n📋 Paramètres DH publics :")
print(f"Générateur g : {g}")
print(f"Premier p (2048 bits) : {p}")
print(f"\nCes paramètres sont publics et peuvent être réutilisés.")

In [ ]:
def diffie_hellman_exchange():
    """
    Simulation d'un échange DH entre Alice et Bob.
    """
    print("=" * 70)
    print("ÉCHANGE DE CLÉS DIFFIE-HELLMAN")
    print("=" * 70)
    
    # Alice génère sa paire de clés
    alice_private_key = parameters.generate_private_key()
    alice_public_key = alice_private_key.public_key()
    
    # Bob génère sa paire de clés
    bob_private_key = parameters.generate_private_key()
    bob_public_key = bob_private_key.public_key()
    
    # Affichage des clés publiques (A = g^a, B = g^b)
    alice_public_bytes = alice_public_key.public_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PublicFormat.SubjectPublicKeyInfo
    )
    bob_public_bytes = bob_public_key.public_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PublicFormat.SubjectPublicKeyInfo
    )
    
    print("\n👩 Alice génère :")
    print(f"   Clé secrète : a (gardée secrète)")
    print(f"   Clé publique : A = g^a (envoyée à Bob)")
    print(f"   Taille : {len(alice_public_bytes)} bytes")
    
    print("\n👨 Bob génère :")
    print(f"   Clé secrète : b (gardée secrète)")
    print(f"   Clé publique : B = g^b (envoyée à Alice)")
    print(f"   Taille : {len(bob_public_bytes)} bytes")
    
    # Alice calcule le secret partagé : K = B^a
    alice_shared_key = alice_private_key.exchange(bob_public_key)
    
    # Bob calcule le secret partagé : K = A^b
    bob_shared_key = bob_private_key.exchange(alice_public_key)
    
    print("\n🔑 Calcul du secret partagé :")
    print(f"   Alice calcule : K = B^a = g^(ab)")
    print(f"   Bob calcule   : K = A^b = g^(ab)")
    
    # Vérification
    assert alice_shared_key == bob_shared_key, "ERREUR : Clés différentes !"
    
    print(f"\n✅ Secret partagé identique : {alice_shared_key.hex()[:64]}...")
    print(f"   Taille : {len(alice_shared_key)} bytes")
    
    # Dérivation de clé (bonne pratique)
    kdf = HKDF(
        algorithm=hashes.SHA256(),
        length=32,
        salt=None,
        info=b'handshake data',
        backend=default_backend()
    )
    derived_key = kdf.derive(alice_shared_key)
    
    print(f"\n🔐 Clé dérivée (HKDF-SHA256) : {derived_key.hex()}")
    print(f"   Cette clé peut être utilisée pour AES-GCM")
    
    return alice_shared_key

shared_secret = diffie_hellman_exchange()

## 2. Sécurité : Problème du logarithme discret

**Attaquant passif** (Eve écoute) :
- Observe : $g, p, A = g^a, B = g^b$
- Veut calculer : $K = g^{ab}$

**Problème Computational DH (CDH)** : Calculer $g^{ab}$ depuis $(g, g^a, g^b)$ est difficile si DLP est difficile.

**Meilleure attaque connue** : Algorithmes sous-exponentiels (ex: Number Field Sieve)
- Complexité : $\exp(O((\log p)^{1/3}))$
- Pour $p$ de 2048 bits : $\approx 2^{112}$ opérations (infaisable)
- Pour $p$ de 3072 bits : $\approx 2^{128}$ opérations (recommandé)

## 3. Attaque Man-in-the-Middle (MITM)

**Problème** : DH seul n'authentifie PAS les participants !

**Scénario** :
1. Alice envoie $A = g^a$ → Eve intercepte
2. Eve envoie $E_1 = g^e$ à Bob (se faisant passer pour Alice)
3. Bob envoie $B = g^b$ → Eve intercepte
4. Eve envoie $E_2 = g^{e'}$ à Alice (se faisant passer pour Bob)
5. Alice partage $K_1 = g^{ae'}$ avec Eve
6. Bob partage $K_2 = g^{eb}$ avec Eve
7. Eve déchiffre/rechiffre tous les messages !

In [ ]:
def mitm_attack_demo():
    """
    Démonstration d'attaque MITM sur DH non authentifié.
    """
    print("=" * 70)
    print("ATTAQUE MAN-IN-THE-MIDDLE sur DH")
    print("=" * 70)
    
    # Alice et Bob
    alice_private = parameters.generate_private_key()
    bob_private = parameters.generate_private_key()
    
    # Eve (attaquante)
    eve_private_1 = parameters.generate_private_key()
    eve_private_2 = parameters.generate_private_key()
    
    print("\n1️⃣ Alice envoie A = g^a")
    alice_public = alice_private.public_key()
    print("   ❌ Eve intercepte !")
    
    print("\n2️⃣ Eve envoie E₁ = g^e₁ à Bob (prétend être Alice)")
    eve_public_to_bob = eve_private_1.public_key()
    
    print("\n3️⃣ Bob envoie B = g^b")
    bob_public = bob_private.public_key()
    print("   ❌ Eve intercepte !")
    
    print("\n4️⃣ Eve envoie E₂ = g^e₂ à Alice (prétend être Bob)")
    eve_public_to_alice = eve_private_2.public_key()
    
    # Calcul des secrets partagés
    alice_thinks_shared_with_bob = alice_private.exchange(eve_public_to_alice)
    eve_shared_with_alice = eve_private_2.exchange(alice_public)
    
    bob_thinks_shared_with_alice = bob_private.exchange(eve_public_to_bob)
    eve_shared_with_bob = eve_private_1.exchange(bob_public)
    
    print("\n🔑 Secrets partagés :")
    print(f"   Alice ↔ Eve : {alice_thinks_shared_with_bob.hex()[:32]}...")
    print(f"   Eve ↔ Alice : {eve_shared_with_alice.hex()[:32]}... (identique ✅)")
    print(f"   Bob ↔ Eve   : {bob_thinks_shared_with_alice.hex()[:32]}...")
    print(f"   Eve ↔ Bob   : {eve_shared_with_bob.hex()[:32]}... (identique ✅)")
    
    assert alice_thinks_shared_with_bob == eve_shared_with_alice
    assert bob_thinks_shared_with_alice == eve_shared_with_bob
    assert alice_thinks_shared_with_bob != bob_thinks_shared_with_alice
    
    print("\n❌ Alice et Bob pensent communiquer ensemble,")
    print("   mais chacun partage une clé DIFFÉRENTE avec Eve !")
    print("\n⚠️  Eve peut déchiffrer tous les messages et les rechiffrer.")
    print("\n✅ SOLUTION : Authenticated DH (avec signatures ou certificats)")

mitm_attack_demo()

## 4. Elliptic Curve Diffie-Hellman (ECDH)

**Avantages** :
- Clés plus courtes (256 bits ECC ≈ 3072 bits DH)
- Calculs plus rapides
- Même niveau de sécurité

**Courbes standard** :
- **Curve25519** (X25519) : recommandé, résistant aux side-channels
- secp256k1 : utilisé dans Bitcoin
- NIST P-256, P-384, P-521

In [ ]:
def ecdh_exchange():
    """
    Échange ECDH avec Curve25519 (X25519).
    """
    print("=" * 70)
    print("ÉCHANGE ECDH avec Curve25519")
    print("=" * 70)
    
    # Alice génère sa paire de clés (courbe elliptique)
    alice_private = ec.generate_private_key(ec.SECP256R1(), default_backend())
    alice_public = alice_private.public_key()
    
    # Bob génère sa paire de clés
    bob_private = ec.generate_private_key(ec.SECP256R1(), default_backend())
    bob_public = bob_private.public_key()
    
    print("\n👩 Alice génère :")
    print(f"   Clé privée : a (256 bits, secrète)")
    print(f"   Clé publique : A = a·G (point sur courbe elliptique)")
    
    print("\n👨 Bob génère :")
    print(f"   Clé privée : b (256 bits, secrète)")
    print(f"   Clé publique : B = b·G (point sur courbe elliptique)")
    
    # Calcul du secret partagé
    alice_shared = alice_private.exchange(ec.ECDH(), bob_public)
    bob_shared = bob_private.exchange(ec.ECDH(), alice_public)
    
    assert alice_shared == bob_shared
    
    print("\n🔑 Secret partagé :")
    print(f"   K = a·B = b·A = (ab)·G")
    print(f"   Valeur : {alice_shared.hex()}")
    print(f"   Taille : {len(alice_shared)} bytes (32 bytes = 256 bits)")
    
    # Comparaison avec DH classique
    print("\n📊 ECDH vs DH classique :")
    print(f"   ECDH (secp256r1)  : Clés publiques ~65 bytes, sécurité ~128 bits")
    print(f"   DH (2048 bits)    : Clés publiques ~256 bytes, sécurité ~112 bits")
    print(f"   ✅ ECDH est plus efficace !")

ecdh_exchange()

## 5. Authenticated Diffie-Hellman

**Solutions** pour prévenir MITM :

1. **Station-to-Station (STS)** : Signer les clés publiques
2. **Certificats** : PKI (TLS/SSL)
3. **Pre-shared keys** : Authentifier avec clé partagée au préalable
4. **Password-Authenticated Key Exchange (PAKE)** : SRP, OPAQUE

In [ ]:
# À COMPLÉTER : Exemple d'Authenticated DH avec signatures
# Nécessite clés de signature (RSA ou ECDSA)

print("Authenticated DH :")
print("1. Alice signe sa clé publique DH avec sa clé de signature")
print("2. Bob vérifie la signature avec la clé publique d'Alice (certifiée)")
print("3. Bob signe sa clé publique DH")
print("4. Alice vérifie la signature de Bob")
print("5. Échange DH sécurisé contre MITM ✅")

## Conclusion

**Points clés** :
- DH permet d'établir un secret partagé sans canal sécurisé préalable
- Basé sur la difficulté du problème du logarithme discret
- ❌ Vulnérable à MITM si pas d'authentification
- ✅ ECDH est plus efficace (clés courtes, rapide)
- En pratique : Utiliser Authenticated DH (TLS, SSH, Signal)

**Applications** :
- TLS 1.3 : (EC)DHE pour forward secrecy
- Signal Protocol : X3DH (Extended Triple DH)
- WireGuard VPN : Curve25519
- SSH : ecdh-sha2-nistp256